In [14]:
# Libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import display
from io import StringIO
import csv
import gzip
from unidecode import unidecode
import html5lib
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine


def ranking_scrape(url):

    # Fetch the url content
    response = requests.get(url)

    # Parse the url content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html5lib')
    def remove_accents(text):
        return unidecode(text) if isinstance(text, str) else text

    # Iterate through all text elements in the HTML and replace accents
    for element in soup.find_all(string=True):
        element.replace_with(remove_accents(element))

    # Find the h2 with id="Histórico"
    h2_header = soup.find('h2', {'id': 'Classificação_geral'})
    desired_table = None
    next_div = None

    if h2_header:
        parent_div = h2_header.find_parent('div')
        next_div = parent_div.find_next_sibling()

    if next_div:
        if next_div.name == "table":
            desired_table = next_div
        else:
            desired_table = next_div.find("table", recursive=False)
    
    if desired_table:

        #Parsing html table to DataFrame
        html_to_table = pd.read_html(StringIO(str(desired_table)))
        Ranking = html_to_table[0]

        # Normalising column name
        if 'Porcentagem/ Pontos' in Ranking.columns: 
            Ranking = Ranking.rename(columns={'Porcentagem/ Pontos':'Porcent_dos_votos'})
        if 'Porcentagem/ Votos' in Ranking.columns: 
            Ranking = Ranking.rename(columns={'Porcentagem/ Votos':'Porcent_dos_votos'})
        Ranking = Ranking.rename(columns={'% dos votos':'Porcent_dos_votos'})

        # Adding the year of the current file
        Ranking['Edicao'] = url.rsplit('_', 1)[-1]

        # Ensuring the Pos. column is always a string
        Ranking['Pos'] = Ranking['Pos.'].astype(str)
        Ranking = Ranking.drop("Pos.", axis=1)

        # Remove double ranking - take only first number
        Ranking['Pos'] = Ranking['Pos'].apply(lambda x: x.split('-')[0] if '-' in x else x)
        

        # Remove Notes number from % Votes and Meio de indicacao
        Ranking['Porcent_dos_votos'] = Ranking['Porcent_dos_votos'].str.replace(r'\[.*', '', regex=True)
        Ranking['Meio de indicacao'] = Ranking['Meio de indicacao'].str.replace(r'\[.*', '', regex=True)

        # Breakdown Meio de Indicacao into two columns: Meio and Nominated by
        Ranking['Indicado por'] = Ranking['Meio de indicacao'].str.extract(r'\((.*?)\)')
        Ranking['Indicado por'] = Ranking['Indicado por'].fillna(Ranking['Meio de indicacao'])
        Ranking['Meio de indicacao'] = Ranking['Meio de indicacao'].str.replace(r'\((.*?)\)', '', regex=True)

        # Some % Votes isn't %, it says "disqualified" or "withdrawn" in a merged cell.
        Ranking['Porcent_dos_votos'] = Ranking.apply(lambda row: row['Porcent_dos_votos'].replace(row['Porcent_dos_votos'], row['Meio de indicacao']) if '%' not in row['Porcent_dos_votos'] else row['Porcent_dos_votos'], axis=1)
        
        # Replace -- in Eliminado em by Finalista
        Ranking['Eliminado em'] = Ranking.apply(lambda row: row['Eliminado em'].replace('--', row['Meio de indicacao']) if '--' in row['Eliminado em'] else row['Eliminado em'], axis=1)

        # Replacing spaces and commas, etc
        Ranking.columns = [col.replace(' ', '_') for col in Ranking.columns]
        Ranking["Porcent_dos_votos"] = Ranking["Porcent_dos_votos"].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
        Ranking = Ranking.replace(",", "_", regex=True)
        
    
    return Ranking

# Appending the Rankings to one single dataframe

base_url = "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_"
number_of_shows = 25

urls = [f"{base_url}{i}" for i in range(1, number_of_shows + 1)]

Combined_ranking = []

for url in urls:
    try:
        ranking_new = ranking_scrape(url)
        Combined_ranking.append(ranking_new)
        print(f"Ranking information for {url} appended")
    except Exception as e:
        print(f"Error processing {url}: {e}")

# Save to csv
Ranking = pd.concat(Combined_ranking, ignore_index=True)

Ranking.to_csv(f'ranking')



print(Ranking["Pos"].unique())  # Inspect unique values in Pos.
display(Ranking.info())
display(Ranking)

Ranking information for https://pt.wikipedia.org/wiki/Big_Brother_Brasil_1 appended
Ranking information for https://pt.wikipedia.org/wiki/Big_Brother_Brasil_2 appended
Ranking information for https://pt.wikipedia.org/wiki/Big_Brother_Brasil_3 appended
Ranking information for https://pt.wikipedia.org/wiki/Big_Brother_Brasil_4 appended
Ranking information for https://pt.wikipedia.org/wiki/Big_Brother_Brasil_5 appended
Ranking information for https://pt.wikipedia.org/wiki/Big_Brother_Brasil_6 appended
Ranking information for https://pt.wikipedia.org/wiki/Big_Brother_Brasil_7 appended
Ranking information for https://pt.wikipedia.org/wiki/Big_Brother_Brasil_8 appended
Ranking information for https://pt.wikipedia.org/wiki/Big_Brother_Brasil_9 appended
Ranking information for https://pt.wikipedia.org/wiki/Big_Brother_Brasil_10 appended
Ranking information for https://pt.wikipedia.org/wiki/Big_Brother_Brasil_11 appended
Ranking information for https://pt.wikipedia.org/wiki/Big_Brother_Brasil_1

None

,Participante,Meio_de_indicacao,Porcent_dos_votos,Eliminado_em,Edicao,Pos,Indicado_por
0,Kleber Bambam,Finalista,68%,Finalista,1,1,Finalista
1,Vanessa Pascale,Finalista,21%,Finalista,1,2,Finalista
2,Andre Gabeh,Finalista,11%,Finalista,1,3,Finalista
3,Sergio Tavares,Lider,52%,Semana 9,1,4,Andre
4,Alessandra Begliomini,Lider,73%,Semana 8,1,5,Sergio
...,...,...,...,...,...,...,...
463,Marcelo Prata,Lideres,55.95%,Semana 1,25,23,Aline & Vinicius
464,Nicole Oliveira,Disputa pelas ultimas vagas,19.86%,Semana 0,25,25,Disputa pelas ultimas vagas
465,Paula Oliveira,Disputa pelas ultimas vagas,19.86%,Semana 0,25,25,Disputa pelas ultimas vagas
466,Cleber Santana,Disputa pelas ultimas vagas,13.74%,Semana 0,25,25,Disputa pelas ultimas vagas


In [15]:
# Load environment variables from .env file
env_path = os.path.abspath("credentials.env")

# Load explicitly
load_dotenv(dotenv_path=env_path, override=True)

# Retrieve values
username = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_HOST')
port = os.getenv('POSTGRES_PORT')
database = os.getenv('POSTGRES_DB')
schema = os.getenv('POSTGRES_SCHEMA')

print("Username:", username)
print("Host:", host)
print("Port:", port)
print("Database:", database)
print("Schema:", schema)

Username: andrea_user
Host: localhost
Port: 5432
Database: bigbrotherbrasil
Schema: wikipedia


In [16]:
# Build the engine
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

# Write DataFrame to Postgres
Ranking.to_sql('ranking', con=engine, schema=schema, if_exists='replace', index=False)
print("DataFrame written to Postgres successfully!")

DataFrame written to Postgres successfully!
